In [1]:
# Dependencies and Setup

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key city py 
from api_keys import weather_api_key
from citipy import citipy

output_data_file = "../Resources/2020_WeaterDataIncluded.csv"
file_path = "../Resources/Vertical_Merge_LatLng.csv"

ModuleNotFoundError: No module named 'api_keys'

In [ ]:
print(weather_api_key)

In [ ]:
df = pd.read_csv(file_path)

In [ ]:
df.head()

In [ ]:
lat_lngs = []
cities = []

lats = df["Latitude"]
lngs = df["Longitude"]
lat_lngs = zip(lats, lngs)
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

len(cities)

In [ ]:

temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities)):
    #get the city
    city = cities[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
            
    elif response.status_code == 404:
        temps.append(np.nan)
        feel_likes.append(np.nan)
        humids.append(np.nan)
        pressures.append(np.nan)
        speeds.append(np.nan)
        cloudiness.append(np.nan)
        cities_found.append(np.nan)
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df["City"] = cities_found
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = cloudiness

df.head(10)

In [ ]:
df

In [ ]:
dropped_df = df.dropna()
dropped_df

In [ ]:
dropped_df.to_csv(output_data_file, index=False)

In [ ]:
plt.scatter(df["Temperature"], df["Happiness Score"], marker="o")

# Incorporate the other graph properties
plt.title("Temperature vs Happiness Score")
plt.ylabel("Happiness Score")
plt.xlabel("Temperature")
plt.grid(True)


# Show plot
plt.show()

In [ ]:
plt.scatter(df["Humidity"], df["Happiness Score"], marker="o")

# Incorporate the other graph properties
plt.title("Humidity vs Happiness Score")
plt.ylabel("Humidity")
plt.xlabel("Happiness Score")
plt.grid(True)


# Show plot
plt.show()